In [ ]:
# 7_optional_pytorch.ipynb
# Reimplementación del modelo extendido en PyTorch

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from torchvision import transforms
import time
from utils.dataloader import load_data_npy
from utils.model_utils import save_model_and_history

In [ ]:
# Configurar dispositivo
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

def preprocesse_images:
   return np.transpose(images, (0, 3, 1, 2))  # NHWC → NCHW 

images_train, categories_train, images_val, categories_val, images_test, categories_test = load_data_npy()


# Normalizar y transponer a formato CHW (PyTorch)
# Transpose to PyTorch format
images_train = preprocess_images(images_train)
images_val = preprocess_images(images_val)
images_test = preprocess_images(images_test)

# Dataset personalizado
class ImageDataset(Dataset):
    def __init__(self, images, labels):
        self.images = torch.tensor(images)
        self.labels = torch.tensor(labels).long()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Crear datasets y dataloaders
train_ds = ImageDataset(images_train, categories_train)
val_ds = ImageDataset(images_val, categories)
test_ds = ImageDataset(images_test, categories_test)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=32)
test_dl = DataLoader(test_ds, batch_size=32)

# --- Definición del modelo CNN extendido ---
class CNNExtended(nn.Module):
    def __init__(self):
        super(CNNExtended, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 18 * 18, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 6)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Inicializar modelo
model = CNNExtended().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- Entrenamiento ---
def train_model(model, train_dl, val_dl, epochs=15):
    best_val_acc = 0.0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_dl:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total

        # Validación
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_dl:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total

        print(f"Epoch {epoch+1}: Loss {running_loss:.4f} | Train Acc {train_acc:.4f} | Val Acc {val_acc:.4f}")
        best_val_acc = max(best_val_acc, val_acc)
    return best_val_acc

# Ejecutar entrenamiento
best_val_accuracy = train_model(model, train_dl, val_dl)
print(f"\nMejor validación accuracy: {best_val_accuracy:.4f}")

# --- Evaluación en test ---
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_dl:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total
test_loss = test_loss / len(test_dl)

print(f"\nTest Accuracy: {test_acc:.4f}  |  Test Loss: {test_loss:.4f}")